<span style="color: red; font-size: 30px; font-weight: bold; background-color: yellow;">
1. Install All the Required Packages
</span>


In [45]:
!nvidia-smi

Sat Mar 15 13:45:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   77C    P0             34W /   70W |    1237MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [46]:
!pip -q install langchain
!pip -q install bitsandbytes accelerate transformers
!pip -q install datasets loralib sentencepiece
!pip -q install pypdf
!pip -q install sentence_transformers

In [47]:
!pip -q install unstructured 

In [48]:
!pip install tokenizers 

In [49]:
!pip install xformers

  Using cached torch-2.6.0-cp310-cp310-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached triton-3.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.4 kB)
Using cached torch-2.6.0-cp310-cp310-manylinux1_x86_64.whl (766.7 MB)
Using cached triton-3.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (253.1 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.0.0
    Uninstalling triton-2.0.0:
      Successfully uninstalled triton-2.0.0
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.18 requires torch<2.6,>=1.10, but you have torch 2.6.0 which is incompatible.
torchaudio 2.5.1+cu121 requires torch==2.5.1, but you have torch 2.6.

In [50]:
!pip install pinecone_client

<span style="color: red; font-size: 30px; font-weight: bold; background-color: yellow;">
2, Import All the Required Libraries
</span>


In [51]:
pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.


In [52]:
!pip install pinecone --quiet

In [53]:
import pinecone

In [54]:
!pip install torch==2.0.1 torchvision==0.15.2 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.0.1%2Bcu118-cp310-cp310-linux_x86_64.whl (2267.3 MB)
  Using cached https://download.pytorch.org/whl/triton-2.0.0-1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (63.3 MB)
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0
    Uninstalling torch-2.6.0:
      Successfully uninstalled torch-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-lightning 2.5.0.post0 requires torch>=2.1.0, but you have torch 2.0.1+cu118 which is incompatible.
torchaudio 2.5.1+cu121 requires torch==2.5.1, but you have torch 2.0.1+cu118 which is incompatible.
xformers 0.0

In [55]:
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
# from langchain.embeddings import OpenAIEmbeddings
# from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Pinecone
import pinecone
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import textwrap
import sys
import os
import torch

In [56]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

<span style="color: red; font-size: 30px; font-weight: bold; background-color: yellow;">
3: Pass the URLs and extract the data from these URLs
</span>


In [57]:
URLs=[
    'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb',
    'https://www.mosaicml.com/blog/mpt-7b',
    'https://stability.ai/blog/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models',
    'https://lmsys.org/blog/2023-03-30-vicuna/'

]

In [58]:
loaders = UnstructuredURLLoader(urls=URLs)

In [59]:
data = loaders.load()

In [60]:
data 

[Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Open in app\n\nSign up\n\nSign in\n\nWrite\n\nSign up\n\nSign in\n\nGoPenAI\n\nHome\n\nAbout\n\nFollow publication\n\nGoPenAI\n\nWhere the ChatGPT community comes together to share insights and stories.\n\nFollow publication\n\nPaper Review\n\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\n\nLlama 2: one of the best open source models\n\nAndrew Lukyanenko\n\nGoPenAI\n\nAndrew Lukyanenko\n\nFollow\n\nPublished in\n\nGoPenAI\n\n15 min read\n\nJul 20, 2023\n\n--\n\nProject link\n\nModel link\n\nPaper link\n\nThe authors of the work present Llama 2, an assortment of pretrained and fine-tuned large language models (LLMs) with sizes varying from 7 billion to 70 billion parameters. The fine-tuned versions, named Llama 2-Chat, are specifically designed for dialogue applications. These models surpass the performance of existing

In [61]:
len(data
   )

4

<span style="color: red; font-size: 30px; font-weight: bold; background-color: yellow;">
4: Split the text into Chunks
</span>


In [62]:
text_splitter = CharacterTextSplitter(separator='\n',
                                     chunk_size=1000,
                                     chunk_overlap=200)

In [63]:
text_chunks = text_splitter.split_documents(data)

In [64]:
len(text_chunks)

88

In [65]:
text_chunks

[Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Open in app\nSign up\nSign in\nWrite\nSign up\nSign in\nGoPenAI\nHome\nAbout\nFollow publication\nGoPenAI\nWhere the ChatGPT community comes together to share insights and stories.\nFollow publication\nPaper Review\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\nLlama 2: one of the best open source models\nAndrew Lukyanenko\nGoPenAI\nAndrew Lukyanenko\nFollow\nPublished in\nGoPenAI\n15 min read\nJul 20, 2023\n--\nProject link\nModel link\nPaper link'),
 Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Llama 2: one of the best open source models\nAndrew Lukyanenko\nGoPenAI\nAndrew Lukyanenko\nFollow\nPublished in\nGoPenAI\n15 min read\nJul 20, 2023\n--\nProject link\nModel link\nPaper link\nThe authors of the work present

<span style="color: red; font-size: 30px; font-weight: bold; background-color: yellow;">
5.Download the HuggingFace Embeddings
</span>


In [66]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = " "

In [67]:
embeddings = HuggingFaceEmbeddings()

<ipython-input-67-d0c9174021d8>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [68]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [69]:
query_result = embeddings.embed_query("How are you")

In [70]:
len(query_result)

768

In [71]:
query_result

[0.024816831573843956,
 0.04983454570174217,
 0.0029632258228957653,
 -0.020911986008286476,
 0.01575610227882862,
 0.0008427164284512401,
 -0.038867294788360596,
 -0.0012208116240799427,
 0.01734468713402748,
 -0.012650596909224987,
 -0.03341116011142731,
 -0.021266309544444084,
 0.0026828551199287176,
 0.013602642342448235,
 0.0033968023490160704,
 -0.053298305720090866,
 -0.01398317702114582,
 -0.057532500475645065,
 -0.044286858290433884,
 -0.014237090945243835,
 -0.05664319545030594,
 0.011137358844280243,
 0.027455981820821762,
 -0.002755955094471574,
 0.05140530318021774,
 -0.0021438701078295708,
 0.029279302805662155,
 -0.0008696034783497453,
 -0.011007783934473991,
 0.048205915838479996,
 -0.02872356027364731,
 0.02568707801401615,
 0.014495937153697014,
 -0.0023129985202103853,
 1.6342876278940821e-06,
 0.06020195409655571,
 -0.02678602933883667,
 -0.03696710243821144,
 0.0686761662364006,
 -0.010282885283231735,
 0.020827004685997963,
 -0.08207923173904419,
 0.00636715581640

<span style="color: red; font-size: 30px; font-weight: bold; background-color: yellow;">
6. Convert the Text Chunks into Embeddings and Create a Knowledge Base
</span>


In [72]:
import pinecone

In [73]:
from pinecone import Pinecone

In [74]:
!pip -q install langchain_pinecone

In [75]:
pinecone_api_key = " "

In [76]:
os.environ['PINECONE_API_KEY'] = pinecone_api_key

In [77]:
index_name = "demo"

In [78]:
from langchain_pinecone import PineconeVectorStore
vectorstore_from_docs = PineconeVectorStore.from_documents(
        text_chunks,
        index_name=index_name,
        embedding=embeddings
    )

<span style="color: red; font-size: 30px; font-weight: bold; background-color: yellow;">
7. Create a Large Language Model (LLM) Wrapper
</span>


In [79]:
pip install langchain langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 4.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [82]:
import os
from langchain_groq import ChatGroq

# Set the Groq API Key
os.environ["GROQ_API_KEY"] = " "

# Load the Llama-2 model from Groq
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0.7)

In [83]:
llm.predict("Please provide a concise summary of the Book Harry Potter")

"The Harry Potter series, written by J.K. Rowling, follows the journey of its titular character, Harry Potter, a young wizard who discovers his magical heritage on his eleventh birthday. The story unfolds as Harry attends Hogwarts School of Witchcraft and Wizardry, where he makes friends with Ron Weasley and Hermione Granger.\n\nThroughout the series, Harry and his friends become entangled in a battle between good and evil, as they confront the dark wizard, Lord Voldemort, who murdered Harry's parents and seeks to dominate the wizarding world. Along the way, Harry uncovers hidden secrets about his past, his parents, and his own destiny.\n\nThe series consists of seven books, each representing a year in Harry's life at Hogwarts, as he navigates adolescence, friendship, and the challenges of the wizarding world. Ultimately, the story culminates in a final showdown between Harry and Voldemort, with the fate of the wizarding world hanging in the balance.\n\nKey themes in the series include

In [84]:
from langchain.chains import RetrievalQA

In [85]:
query = "How good is Vicuna?"

In [86]:
docs = vectorstore_from_docs.similarity_search(query, k=3)

In [87]:
docs

[Document(id='c7b32a78-f1f2-473e-924d-c4eb2d97ac36', metadata={'source': 'https://lmsys.org/blog/2023-03-30-vicuna/'}, page_content='Vicuna (generated by stable diffusion 2.1)\nAccording to a fun and non-scientific evaluation with GPT-4. Further rigorous evaluation is needed.\nHow Good is Vicuna?\nAfter fine-tuning Vicuna with 70K user-shared ChatGPT conversations, we discover that Vicuna becomes capable of generating more detailed and well-structured answers compared to Alpaca (see examples below), with the quality on par with ChatGPT.'),
 Document(id='e2aa149d-e35f-48da-b211-435593213b84', metadata={'source': 'https://lmsys.org/blog/2023-03-30-vicuna/'}, page_content='LMSYS ORG\nProjectsBlogAboutDonationsChatbot Arena\nVicuna: An Open-Source Chatbot Impressing GPT-4 with 90%* ChatGPT Quality\nby: The Vicuna Team, Mar 30, 2023\nWe introduce Vicuna-13B, an open-source chatbot trained by fine-tuning LLaMA on user-shared conversations collected from ShareGPT. Preliminary evaluation using

In [88]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore_from_docs.as_retriever())

In [89]:
query = "How good is Vicuna?"
qa.run(query)

<ipython-input-89-8e616d530869>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run(query)


"Vicuna is considered to be of high quality, with its performance comparable to ChatGPT. According to evaluations using GPT-4 as a judge, Vicuna achieves more than 90% quality of OpenAI ChatGPT and Google Bard, and outperforms other models like LLaMA and Stanford Alpaca in more than 90% of cases. In 45% of questions, GPT-4 rates Vicuna's response as better or equal to ChatGPT's. Vicuna's total score is 92% of ChatGPT's, based on a quantitative score assigned by GPT-4. However, it's worth noting that Vicuna still has limitations, such as struggling with tasks involving reasoning, mathematics, and ensuring factual accuracy."

In [90]:
query = "How does Llama 2 outperforms other models"
qa.run(query)

"According to the context, Llama 2 outperforms other models in the following ways:\n\n1. Llama 2 models with 7 billion and 30 billion parameters outdo MPT models of similar size in all categories except code benchmarks.\n2. Llama 2's 7 billion and 34 billion parameter models outperform the 7 billion and 40 billion parameter Falcon models in all benchmark categories.\n3. The Llama 2 70B model surpasses all open-source models.\n4. The Llama 2 70B model performs similarly to the closed-source GPT-3.5 on the MMLU and GSM8K benchmarks.\n5. The Llama 2 70B model matches or exceeds the performance of PaLM (540 billion parameters) on nearly all benchmarks.\n\nHowever, it's noted that there is still a significant performance gap between the Llama 2 70B model and both GPT-4 and PaLM-2-L, particularly in coding benchmarks.\n\nAdditionally, the Llama 2 models have shown significant improvement over their Llama 1 counterparts, with the 70 billion-parameter Llama 2 model improving results on the MML

In [91]:
query = "What is is stableLM?"
qa.run(query)

'Stable LM is an open-source language model developed by Stability AI. It is available in various sizes, including 3 billion and 7 billion parameters, with larger models (15 billion to 65 billion parameters) to be released in the future. The model can generate text and code, and is designed to be used for commercial or research purposes, subject to the terms of the CC BY-SA-4.0 license.'

In [92]:
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Input Prompt:  what mpt 7b


<ipython-input-92-c213a85929f6>:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({'query': user_input})


Answer: MPT-7B appears to be a type of artificial intelligence (AI) model, specifically a large language model. The "7B" in its name suggests that it has 7 billion parameters, which is a measure of the model's complexity and capacity to learn.

From the context, it seems that MPT-7B is a model developed by the MosaicML NLP team, and it is designed to be fast, easy, and cheap to deploy for inference. It is compatible with the HuggingFace ecosystem and can be used for tasks such as generating text.

There are also finetuned versions of the model, such as MPT-7B-Instruct, which are tailored for specific tasks or applications. The model was trained on the MosaicML platform using a combination of tools and technologies, including PyTorch, Composer, and LLM Foundry.


Input Prompt:  exit


Exiting


SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
